In [3]:
import pandas as pd
import openpyxl
import json
import requests
import matplotlib.pyplot as plt
import numpy as np
from pymongo import MongoClient

#### Read ticker symbols of stocks from etoro sheet (exported to another document)

In [6]:
file = r'D:\\lianz\Desktop\\Python\personal_projects\\personal_finance\\tickers.json'
with open(file, 'r+') as f:
    tickers = json.load(f)

In [7]:
tickers[0]

'A'

#### Read in json files (requests not made repetitively)

In [13]:
def read_json(ticker_list, statement):

    with open(f'D:\lianz\Desktop\Python\personal_projects\personal_finance\{statement}/{ticker_list}.json', 'r+') as json_file:
        x = json.load(json_file)

    return x

In [11]:
company_statements = ['income-statement',
                      'cash-flow-statement', 'balance-sheet-statement']

In [14]:
d = read_json(tickers[0],company_statements[0])

In [10]:
[d[i]['calendarYear'] for i in range(len(d))]

['2022', '2021', '2020', '2019', '2018']

In [15]:
d[0].items()

dict_items([('date', '2022-10-31'), ('symbol', 'A'), ('reportedCurrency', 'USD'), ('cik', '0001090872'), ('fillingDate', '2022-12-21'), ('acceptedDate', '2022-12-20 18:42:30'), ('calendarYear', '2022'), ('period', 'FY'), ('revenue', 6848000000), ('costOfRevenue', 3126000000), ('grossProfit', 3722000000), ('grossProfitRatio', 0.5435163551401869), ('researchAndDevelopmentExpenses', 467000000), ('generalAndAdministrativeExpenses', 0.0), ('sellingAndMarketingExpenses', 0.0), ('sellingGeneralAndAdministrativeExpenses', 1637000000), ('otherExpenses', 0.0), ('operatingExpenses', 2104000000), ('costAndExpenses', 5230000000), ('interestIncome', 9000000.0), ('interestExpense', 84000000), ('depreciationAndAmortization', 317000000), ('ebitda', 1905000000), ('ebitdaratio', 0.27818341121495327), ('operatingIncome', 1618000000), ('operatingIncomeRatio', 0.2362733644859813), ('totalOtherIncomeExpensesNet', -114000000), ('incomeBeforeTax', 1504000000), ('incomeBeforeTaxRatio', 0.21962616822429906), ('i

In [16]:
l = []
dict_holder = {}

for n in ['epsdiluted','netIncome','revenue']:
    # print(n)

    for i,x in enumerate(d[::-1]):
        dict_holder[x['calendarYear']] = x[f'{n}']
    l.append(dict_holder)
    dict_holder = {}
l

[{'2018': 0.97, '2019': 3.37, '2020': 2.3, '2021': 3.94, '2022': 4.08},
 {'2018': 316000000,
  '2019': 1071000000,
  '2020': 719000000,
  '2021': 1210000000,
  '2022': 1254000000},
 {'2018': 4914000000,
  '2019': 5163000000,
  '2020': 5339000000,
  '2021': 6319000000,
  '2022': 6848000000}]

In [15]:
eps_df = pd.DataFrame.from_records(l, index=['epsdiluted','netIncome','revenue']).T
eps_df

NameError: name 'pd' is not defined

In [2]:
terms_interested = {'revenue': 'revenue',
                    'gross margin%': 'grossProfitRatio',
                    'Operating Income': 'operatingIncome',
                    'Operating Margin %': 'operatingIncomeRatio',
                    'Net Income': 'netIncome',
                    'Net Income Margin': 'netIncomeRatio',
                    'Earnings per Share': 'epsdiluted',
                    'Shares Oustanding (diluted)': 'weightedAverageShsOutDil'}

In [13]:
list(terms_interested.keys()).index('gross margin%')

1

In [60]:
metric = [i for i in terms_interested.keys()][:3]
metric

['revenue', 'gross margin%', 'Operating Income']

In [61]:
[terms_interested[i] for i in metric]

['revenue', 'grossProfitRatio', 'operatingIncome']

In [16]:
items_interested = terms_interested.values()

for items in items_interested:
    print(items)

revenue
grossProfitRatio
operatingIncome
operatingIncomeRatio
netIncome
netIncomeRatio
epsdiluted
weightedAverageShsOutDil


In [17]:
[items for items in items_interested if items in d[0].keys()]

['revenue',
 'grossProfitRatio',
 'operatingIncome',
 'operatingIncomeRatio',
 'netIncome',
 'netIncomeRatio',
 'epsdiluted',
 'weightedAverageShsOutDil']

In [18]:
arrangement = (2,3)
column_names = []

for i,nrow in enumerate(arrangement):
    for ncolumn in range(1,nrow+1):
        column_names.append(f'c_{i}_{ncolumn}')

column_names

['c_0_1', 'c_0_2', 'c_1_1', 'c_1_2', 'c_1_3']

In [19]:
eps_df.index

Index(['2018', '2019', '2020', '2021', '2022'], dtype='object')

In [20]:
eps_df


,epsdiluted,netIncome,revenue
2018,0.97,3.160000e+08,4.914000e+09
2019,3.37,1.071000e+09,5.163000e+09
2020,2.30,7.190000e+08,5.339000e+09
2021,3.94,1.210000e+09,6.319000e+09
2022,4.08,1.254000e+09,6.848000e+09


In [19]:
pd.options.display.float_format = '{:.2%}'.format
eps_df['epsdiluted'].pct_change(periods=1).fillna(0)


2018     0.00%
2019   247.42%
2020   -31.75%
2021    71.30%
2022     3.55%
Name: epsdiluted, dtype: float64

In [22]:
type(eps_df.columns[0])

str

In [20]:
import os
import pathlib

In [22]:
arr = os.listdir('D:\lianz\Desktop\Python\personal_projects\personal_finance\\balance-sheet-statement')

In [23]:
list_1 = [
  "UAL",
  "AF.PA",
  "T",
  "JNJ",
  "DIS",
  "AAPL",
  "TSLA",
  "JPM",
  "GOOG",
]

list_2 = [
  "AMD",
  "INGA.NV",
  "SHEL.L",
  "SMSN.L",
  "WMT",
  "NVDA",
  "WBD",
  "MMM",
  "AMZN",
  "COST"
]

In [24]:
[i for i in list_1 if i not in list_2]

['UAL', 'AF.PA', 'T', 'JNJ', 'DIS', 'AAPL', 'TSLA', 'JPM', 'GOOG']

In [4]:
from pymongo import MongoClient,ASCENDING, DESCENDING
from bson.objectid import ObjectId


with open('D:\lianz\Desktop\Python\personal_projects\\finance_dashboard\mongodb_api.txt','r') as f:
    cluster = f.readlines()[0]
    
client = MongoClient(cluster)

# print(client.list_database_names())

db = client.FinanceApp

balance_sheet_collection = db.balance_sheet
income_collection = db.income_statement
cash_collection = db.cash_flow_statement
company_collection = db.company_profile

In [5]:
def generate_key_metrics(financial_statement, list_of_metrics):
    l = []
    dict_holder = {}
    columns = financial_statement[0].keys()

    for n in list_of_metrics:

        if n in columns:
            for i, x in enumerate(financial_statement[::-1]):
                dict_holder[x['calendarYear']] = x[f'{n}']

            l.append(dict_holder)
            dict_holder = {}

        else:
            pass

    df = pd.DataFrame.from_records(
        l, index=[items for items in list_of_metrics if items in columns])

    # df.style.format({f"{df}": "{:,}"})

    return df

In [6]:
def read_statement(collection, ticker):
    
        statement = [i for i in collection.find({'symbol':ticker}).sort('date', DESCENDING)]

        return statement

In [7]:
terms_interested = {'Revenue': 'revenue',
                    'Gross margin%': 'grossProfitRatio',
                    'Operating Income': 'operatingIncome',
                    'Operating Margin %': 'operatingIncomeRatio',
                    'Net Income': 'netIncome',
                    'Net Income Margin': 'netIncomeRatio',
                    'Earnings per Share': 'epsdiluted',
                    'Shares Oustanding (diluted)': 'weightedAverageShsOutDil',
                    'Dividends': 'dividendsPaid',
                    'Operating Cash Flow': 'operatingCashFlow',
                    'Cap Spending': 'capitalExpenditure',
                    'Free Cash Flow': 'freeCashFlow',
                    'Free Cash Flow per Share': 'freeCashFlow / shares outstanding',
                    'Working Capital': 'totalCurrentAssets - totalCurrentLiabilities',
                    'Net Debt': 'netDebt'
                    }

In [8]:
collections = [balance_sheet_collection,income_collection,cash_collection]

In [9]:
concat_df = pd.concat([generate_key_metrics(read_statement(x,'AAPL'), terms_interested.values()) for x in collections],axis=0)

In [10]:
generate_key_metrics(read_statement(cash_collection,'AAPL'), terms_interested.values())

,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
netIncome,454000000,474900000,309800000,530400000,86600000,310178000,424000000,-816000000,-1045000000,309000000,...,37037000000,39510000000,53394000000,45687000000,48351000000,59531000000,55256000000,57411000000,94680000000,99803000000
dividendsPaid,-50300000,-53800000,-56500000,-57200000,-55600000,-56572000,-58000000,-14000000,0,0,...,-10564000000,-11126000000,-11561000000,-12150000000,-12769000000,-13712000000,-14119000000,-14081000000,-14467000000,-14841000000
operatingCashFlow,507300000,963900000,128700000,883700000,-661800000,736995000,-240000000,519000000,188000000,775000000,...,53666000000,59713000000,81266000000,65824000000,63598000000,77434000000,69391000000,80674000000,104038000000,122151000000
capitalExpenditure,-239000000,-224300000,-218300000,-194900000,-213100000,-159587000,-159000000,-67000000,-53000000,-56000000,...,-9076000000,-9813000000,-11488000000,-13548000000,-12795000000,-13313000000,-10495000000,-7309000000,-11085000000,-10708000000
freeCashFlow,268300000,739600000,-89600000,688800000,-874900000,577408000,-399000000,452000000,135000000,719000000,...,44590000000,49900000000,69778000000,52276000000,50803000000,64121000000,58896000000,73365000000,92953000000,111443000000


In [11]:
concat_df.loc[~concat_df.index.duplicated(keep='first'),:]

,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
netDebt,-3.370000e+08,-5.762000e+08,-5.651000e+08,-2.307000e+08,-3.815000e+08,-2.521000e+08,-4.555000e+08,-3.141000e+08,1.468000e+08,-6.072880e+08,...,2.701000e+09,2.145100e+10,4.334200e+10,6.654800e+10,9.539100e+10,8.857000e+10,5.920300e+10,7.442000e+10,8.977900e+10,9.642300e+10
revenue,1.918300e+09,1.901900e+09,2.661100e+09,4.071400e+09,5.284000e+09,5.558400e+09,6.308800e+09,7.086500e+09,7.977000e+09,9.188748e+09,...,1.709100e+11,1.827950e+11,2.337150e+11,2.156390e+11,2.292340e+11,2.655950e+11,2.601740e+11,2.745150e+11,3.658170e+11,3.943280e+11
grossProfitRatio,4.390867e-01,5.583364e-01,5.394010e-01,5.300879e-01,5.136260e-01,5.675914e-01,5.070853e-01,4.674099e-01,3.628306e-01,2.550764e-01,...,3.762448e-01,3.858804e-01,4.005990e-01,3.907596e-01,3.846986e-01,3.834372e-01,3.781777e-01,3.823325e-01,4.177936e-01,4.330963e-01
operatingIncome,1.473000e+08,2.735000e+08,3.715000e+08,6.203000e+08,6.343000e+08,7.120000e+08,4.473000e+08,8.057000e+08,1.104000e+08,5.222740e+08,...,4.899900e+10,5.250300e+10,7.123000e+10,6.002400e+10,6.134400e+10,7.089800e+10,6.393000e+10,6.628800e+10,1.089490e+11,1.194370e+11
operatingIncomeRatio,7.678674e-02,1.438036e-01,1.396039e-01,1.523555e-01,1.200416e-01,1.280944e-01,7.090096e-02,1.136951e-01,1.383979e-02,5.683843e-02,...,2.866948e-01,2.872234e-01,3.047729e-01,2.783541e-01,2.676043e-01,2.669403e-01,2.457202e-01,2.414731e-01,2.978238e-01,3.028874e-01
netIncome,6.120000e+07,1.540000e+08,2.175000e+08,4.003000e+08,4.540000e+08,4.749000e+08,3.098000e+08,5.304000e+08,8.660000e+07,3.101780e+08,...,3.703700e+10,3.951000e+10,5.339400e+10,4.568700e+10,4.835100e+10,5.953100e+10,5.525600e+10,5.741100e+10,9.468000e+10,9.980300e+10
netIncomeRatio,3.190325e-02,8.097166e-02,8.173312e-02,9.831999e-02,8.591976e-02,8.543826e-02,4.910601e-02,7.484654e-02,1.085621e-02,3.375629e-02,...,2.167047e-01,2.161438e-01,2.284577e-01,2.118680e-01,2.109242e-01,2.241420e-01,2.123809e-01,2.091361e-01,2.588179e-01,2.530964e-01
epsdiluted,4.464000e-03,1.071400e-02,1.482100e-02,2.750000e-02,3.160700e-02,3.375000e-02,2.303600e-02,3.875000e-02,6.607000e-03,2.330400e-02,...,1.419643e+00,1.612500e+00,2.305000e+00,2.077500e+00,2.302500e+00,2.977500e+00,2.972500e+00,3.280000e+00,5.610000e+00,6.110000e+00
weightedAverageShsOutDil,1.370880e+10,1.437333e+10,1.467470e+10,1.455636e+10,1.436384e+10,1.407111e+10,1.344868e+10,1.368774e+10,1.310703e+10,1.330672e+10,...,2.608654e+10,2.449065e+10,2.317228e+10,2.200112e+10,2.100677e+10,2.000044e+10,1.859565e+10,1.752821e+10,1.686492e+10,1.632582e+10
dividendsPaid,NaN,NaN,NaN,NaN,-5.030000e+07,-5.380000e+07,-5.650000e+07,-5.720000e+07,-5.560000e+07,-5.657200e+07,...,-1.056400e+10,-1.112600e+10,-1.156100e+10,-1.215000e+10,-1.276900e+10,-1.371200e+10,-1.411900e+10,-1.408100e+10,-1.446700e+10,-1.484100e+10


In [73]:
concat_df.T.dtypes

netDebt                     float64
revenue                     float64
grossProfitRatio            float64
operatingIncome             float64
operatingIncomeRatio        float64
netIncome                   float64
netIncomeRatio              float64
epsdiluted                  float64
weightedAverageShsOutDil    float64
netIncome                   float64
dividendsPaid               float64
operatingCashFlow           float64
capitalExpenditure          float64
freeCashFlow                float64
dtype: object

In [12]:
test = {'c1':'ticker_list_box', 'c2':'ticker_compare'}

In [13]:
for i,j in test.items():
    print(i,j)

c1 ticker_list_box
c2 ticker_compare


In [25]:
def make_pretty(styler, use_on):
    # styler.set_caption("Weather Conditions")
    # styler.format(rain_condition)
    # styler.format_index(lambda v: v.strftime("%A"))
    # styler.background_gradient(axis=None, vmin=1, vmax=5, cmap="YlGnBu")
    styler.applymap(lambda x: 'color:red;' if (x < 0 if type(x) != str else None) else None)
    # styler.highlight_min(color='indianred', axis=0)
    # styler.highlight_max(color='green', axis=0)
    if use_on == 'statements':
        styler.format(precision=0, na_rep='MISSING', thousands=' ',formatter={'grossProfitRatio': "{:.0%}",
                                                                                  'ebitdaratio': "{:.0%}",
                                                                                  'netIncomeRatio': "{:.0%}",
                                                                                  'operatingIncomeRatio': "{:.0%}",
                                                                                  'incomeBeforeTaxRatio': "{:.0%}",
                                                                                  'eps': "{:.2f}",
                                                                                  'epsdiluted': "{:.2f}"
                                                                                 })
    else:
        styler.format(na_rep='-',formatter='{:.0%}')

    return styler

In [42]:
concat_df.loc[~concat_df.index.duplicated(keep='first'),:]

,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
netDebt,-3.370000e+08,-5.762000e+08,-5.651000e+08,-2.307000e+08,-3.815000e+08,-2.521000e+08,-4.555000e+08,-3.141000e+08,1.468000e+08,-6.072880e+08,...,2.701000e+09,2.145100e+10,4.334200e+10,6.654800e+10,9.539100e+10,8.857000e+10,5.920300e+10,7.442000e+10,8.977900e+10,9.642300e+10
revenue,1.918300e+09,1.901900e+09,2.661100e+09,4.071400e+09,5.284000e+09,5.558400e+09,6.308800e+09,7.086500e+09,7.977000e+09,9.188748e+09,...,1.709100e+11,1.827950e+11,2.337150e+11,2.156390e+11,2.292340e+11,2.655950e+11,2.601740e+11,2.745150e+11,3.658170e+11,3.943280e+11
grossProfitRatio,4.390867e-01,5.583364e-01,5.394010e-01,5.300879e-01,5.136260e-01,5.675914e-01,5.070853e-01,4.674099e-01,3.628306e-01,2.550764e-01,...,3.762448e-01,3.858804e-01,4.005990e-01,3.907596e-01,3.846986e-01,3.834372e-01,3.781777e-01,3.823325e-01,4.177936e-01,4.330963e-01
operatingIncome,1.473000e+08,2.735000e+08,3.715000e+08,6.203000e+08,6.343000e+08,7.120000e+08,4.473000e+08,8.057000e+08,1.104000e+08,5.222740e+08,...,4.899900e+10,5.250300e+10,7.123000e+10,6.002400e+10,6.134400e+10,7.089800e+10,6.393000e+10,6.628800e+10,1.089490e+11,1.194370e+11
operatingIncomeRatio,7.678674e-02,1.438036e-01,1.396039e-01,1.523555e-01,1.200416e-01,1.280944e-01,7.090096e-02,1.136951e-01,1.383979e-02,5.683843e-02,...,2.866948e-01,2.872234e-01,3.047729e-01,2.783541e-01,2.676043e-01,2.669403e-01,2.457202e-01,2.414731e-01,2.978238e-01,3.028874e-01
netIncome,6.120000e+07,1.540000e+08,2.175000e+08,4.003000e+08,4.540000e+08,4.749000e+08,3.098000e+08,5.304000e+08,8.660000e+07,3.101780e+08,...,3.703700e+10,3.951000e+10,5.339400e+10,4.568700e+10,4.835100e+10,5.953100e+10,5.525600e+10,5.741100e+10,9.468000e+10,9.980300e+10
netIncomeRatio,3.190325e-02,8.097166e-02,8.173312e-02,9.831999e-02,8.591976e-02,8.543826e-02,4.910601e-02,7.484654e-02,1.085621e-02,3.375629e-02,...,2.167047e-01,2.161438e-01,2.284577e-01,2.118680e-01,2.109242e-01,2.241420e-01,2.123809e-01,2.091361e-01,2.588179e-01,2.530964e-01
epsdiluted,4.464000e-03,1.071400e-02,1.482100e-02,2.750000e-02,3.160700e-02,3.375000e-02,2.303600e-02,3.875000e-02,6.607000e-03,2.330400e-02,...,1.419643e+00,1.612500e+00,2.305000e+00,2.077500e+00,2.302500e+00,2.977500e+00,2.972500e+00,3.280000e+00,5.610000e+00,6.110000e+00
weightedAverageShsOutDil,1.370880e+10,1.437333e+10,1.467470e+10,1.455636e+10,1.436384e+10,1.407111e+10,1.344868e+10,1.368774e+10,1.310703e+10,1.330672e+10,...,2.608654e+10,2.449065e+10,2.317228e+10,2.200112e+10,2.100677e+10,2.000044e+10,1.859565e+10,1.752821e+10,1.686492e+10,1.632582e+10
dividendsPaid,NaN,NaN,NaN,NaN,-5.030000e+07,-5.380000e+07,-5.650000e+07,-5.720000e+07,-5.560000e+07,-5.657200e+07,...,-1.056400e+10,-1.112600e+10,-1.156100e+10,-1.215000e+10,-1.276900e+10,-1.371200e+10,-1.411900e+10,-1.408100e+10,-1.446700e+10,-1.484100e+10


In [38]:
stock_price = [8,7,1,2,3,6,9,12,3,2]
k = 3

In [44]:
count = 0
for i in range(len(stock_price) - k):
    for j,value in enumerate(stock_price[i:k+i]):
        print(value)

8
7
1
7
1
2
1
2
3
2
3
6
3
6
9
6
9
12
9
12
3


In [31]:
len(stock_price)

10

In [8]:
import yfinance as yf

In [9]:
aapl = yf.Ticker('AAPL')

In [ ]:
aapl.info